In [186]:
#!pip install surprise
#!pip install import-ipynb
import os
import csv
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import pickle
from random import *
from surprise import accuracy
from collections import defaultdict
from surprise import KNNBasic
from collections import defaultdict
from operator import itemgetter
import NetflixLoadData as NetflixLoadData 
#sns.set_style("darkgrid")

In [187]:
use_pickle_file = True
max_n = 2500000 #how many rows we want from data_ratings and data_rating_plus_movie_title
reader = Reader(line_format='user item rating', rating_scale=(1, 5))

## Get data

In [188]:
NetflixLoadData.commented_implementation()

This part was a huge problem for me since the original files where weirdly structured
For example the ratings file was structured with the movie_id randomly being placed in a line and each line after that was refering to that movie without having the id in their line, then another movie randomly started in a line and lines after that were a reference to that movie

How it looked before:
1:
 1488844, 3, 2005-09-06 
 822109, 5, 2005-05-13 
 885013, 4, 2005-10-19 
 30878, 4, 2005-12-26 
....
....
...

2:
 2059652, 4, 2005-09-05 
 1666394, 3, 2005-04-19 
 1759415, 4, 2005-04-22 
 1959936, 5, 2005-11-21 
....
....
...

the way I fixed this was include everything in a dataframe and if there was a missing rating then that was a movie_id other wise it was a customer rating, then I combined all these rows to those ratingsm so each movie had a correct reference


(** to see all how these four files below were created, look at the file NetflixLoadData.py and there look at these functions -> get_dataframes(), create_dataframe() and commented_code_that_was_used_before() **)

In [189]:
data_movies, data_rating, data_rating_plus_movie_title, _ = NetflixLoadData.get_data_files(use_small_dataset=True)

In [190]:
data_rating

,movie_id,customer_id,rating
0,1,1488844,3.0
1,1,822109,5.0
2,1,885013,4.0
3,1,30878,4.0
4,1,823519,3.0
...,...,...,...
813413,4499,101554,4.0
813414,4499,1334851,3.0
813415,4499,1852040,1.0
813416,4499,185372,1.0


## Helper functions

In [191]:
def save_to_pickle(name, df):
    path_name = "pickle/"+name+".pickle"
    pickle_file = open(path_name,"wb")
    pickle.dump(df, pickle_file)
    pickle_file.close()

def load_pickle(name):
    path_name = "pickle/"+name+".pickle"
    return_input = open(path_name, "rb")
    return pickle.load(return_input)

## Finding information about the data before doing anything major

In [192]:
import MovieCustomerInformation as information
max_rating = 4
min_rating = 4

### Details about all customers/movies

In [193]:
# get the average movie rating for all customers
# used to determine if this user typically gives bad or good reviews
# and then we can see if he really hates or loves a movie
all_customers_average_ratings =  information.all_average_ratings(df=data_rating, type='customer_id')
print(all_customers_average_ratings)

     customer_id  rating       avg_rating
                     sum count           
0            685   124.0    30   4.133333
1            915    30.0     7   4.285714
2            967    87.0    24   3.625000
3           1333  2007.0   750   2.676000
4           2442   404.0   118   3.423729
...          ...     ...   ...        ...
4824     2646591   163.0    51   3.196078
4825     2647871   814.0   248   3.282258
4826     2648122   306.0    85   3.600000
4827     2648650    83.0    27   3.074074
4828     2648861   181.0    53   3.415094

[4829 rows x 4 columns]


In [194]:
all_movies_average_rating = information.all_average_ratings(df=data_rating, type='movie_id')
print(all_movies_average_rating)

     movie_id  rating       avg_rating
                  sum count           
0           1  2051.0   547   3.749543
1           2   516.0   145   3.558621
2           3  7326.0  2012   3.641153
3           4   389.0   142   2.739437
4           5  4468.0  1140   3.919298
...       ...     ...   ...        ...
4494     4495   162.0    60   2.700000
4495     4496  2676.0   713   3.753156
4496     4497   220.0    84   2.619048
4497     4498    88.0    36   2.444444
4498     4499   230.0    83   2.771084

[4499 rows x 4 columns]


In [195]:
information.get_avg_rating_less_than(df=all_customers_average_ratings , max_rating=max_rating)

     customer_id  rating       avg_rating
                     sum count           
2            967    87.0    24   3.625000
3           1333  2007.0   750   2.676000
4           2442   404.0   118   3.423729
5           3321  2718.0  1062   2.559322
7           3718   529.0   175   3.022857
...          ...     ...   ...        ...
4824     2646591   163.0    51   3.196078
4825     2647871   814.0   248   3.282258
4826     2648122   306.0    85   3.600000
4827     2648650    83.0    27   3.074074
4828     2648861   181.0    53   3.415094

[4039 rows x 4 columns]


In [196]:
information.get_avg_rating_higher_than(df=all_customers_average_ratings, min_rating=min_rating)

     customer_id  rating       avg_rating
                     sum count           
0            685   124.0    30   4.133333
1            915    30.0     7   4.285714
6           3417   165.0    41   4.024390
12          5225   108.0    26   4.153846
18          8117  1062.0   260   4.084615
...          ...     ...   ...        ...
4800     2632269   103.0    25   4.120000
4804     2634414    33.0     8   4.125000
4806     2635437   499.0   115   4.339130
4807     2635895   314.0    75   4.186667
4814     2640085   158.0    34   4.647059

[721 rows x 4 columns]


In [197]:
information.get_avg_rating_less_than(df=all_movies_average_rating, max_rating=max_rating)

     movie_id  rating       avg_rating
                  sum count           
0           1  2051.0   547   3.749543
1           2   516.0   145   3.558621
2           3  7326.0  2012   3.641153
3           4   389.0   142   2.739437
4           5  4468.0  1140   3.919298
...       ...     ...   ...        ...
4494     4495   162.0    60   2.700000
4495     4496  2676.0   713   3.753156
4496     4497   220.0    84   2.619048
4497     4498    88.0    36   2.444444
4498     4499   230.0    83   2.771084

[4457 rows x 4 columns]


In [198]:
information.get_avg_rating_higher_than(df=all_movies_average_rating, min_rating=min_rating)

     movie_id   rating       avg_rating
                   sum count           
12         13     38.0     8   4.750000
222       223   3594.0   875   4.107429
240       241   5359.0  1317   4.069096
570       571  12222.0  3021   4.045680
752       753     33.0     8   4.125000
871       872   5000.0  1206   4.145937
885       886   7719.0  1918   4.024505
1019     1020   3501.0   842   4.157957
1255     1256    925.0   227   4.074890
1475     1476   1936.0   447   4.331096
1624     1625   5149.0  1258   4.093005
1914     1915    456.0   113   4.035398
2101     2102   1714.0   384   4.463542
2141     2142     49.0    11   4.454545
2161     2162   1919.0   471   4.074310
2171     2172   3516.0   811   4.335388
2429     2430   5708.0  1399   4.080057
2451     2452  11777.0  2736   4.304459
2462     2463    206.0    51   4.039216
2607     2608    474.0   116   4.086207
2699     2700    763.0   185   4.124324
2742     2743   6786.0  1695   4.003540
2781     2782   9395.0  2320   4.049569


In [199]:
count_movies_rated_by_customer = information.all_get_rated_count(df=data_rating, type="customer_id")
count_movies_rated_by_customer

,customer_id,movie_id
0,685,30
1,915,7
2,967,24
3,1333,750
4,2442,118
...,...,...
4824,2646591,51
4825,2647871,248
4826,2648122,85
4827,2648650,27


find customers who rated a random movie

In [200]:
movie_to_find = data_movies['movie_title'][randrange(1000)]
information.get_customers_who_rated_movie_title(movie_title=movie_to_find)

,movie_id,customer_id,rating,movie_year,movie_title
168286,898,1331154,3.0,2000.0,The Body
168287,898,1977959,2.0,2000.0,The Body
168288,898,21983,5.0,2000.0,The Body
168289,898,1333,2.0,2000.0,The Body
168290,898,303948,4.0,2000.0,The Body
...,...,...,...,...,...
168376,898,1663888,2.0,2000.0,The Body
168377,898,453043,2.0,2000.0,The Body
168378,898,1639792,1.0,2000.0,The Body
168379,898,1033930,3.0,2000.0,The Body


In [201]:
information.get_avg_rating_for_movie_title(movie_title=movie_to_find)

movie_id rating       avg_rating movie_id                       
                sum count            movie_id movie_year movie_title
897      898  254.0    95   2.673684      898     2000.0    The Body

finding customers who rated a specific movie

In [202]:

information.get_customers_who_rated_movie_title(movie_title="Harold and Kumar Go to White Castle")

,movie_id,customer_id,rating,movie_year,movie_title
51402,290,1436762,4.0,2004.0,Harold and Kumar Go to White Castle
51403,290,1745265,5.0,2004.0,Harold and Kumar Go to White Castle
51404,290,1176140,2.0,2004.0,Harold and Kumar Go to White Castle
51405,290,439011,5.0,2004.0,Harold and Kumar Go to White Castle
51406,290,2632461,4.0,2004.0,Harold and Kumar Go to White Castle
...,...,...,...,...,...
52628,290,999312,5.0,2004.0,Harold and Kumar Go to White Castle
52629,290,520675,4.0,2004.0,Harold and Kumar Go to White Castle
52630,290,1550207,2.0,2004.0,Harold and Kumar Go to White Castle
52631,290,1424777,3.0,2004.0,Harold and Kumar Go to White Castle


In [203]:
information.get_avg_rating_for_movie_title(movie_title="Harold and Kumar Go to White Castle")

movie_id  rating       avg_rating movie_id             \
                 sum count            movie_id movie_year   
289      290  4351.0  1231   3.534525      290     2004.0   

                                          
                             movie_title  
289  Harold and Kumar Go to White Castle

In [204]:
def generate_random_user_ids(from_n, to_n):
    ids = np.unique(data_rating['customer_id'])[from_n:to_n]
    print("random user id used while in development\n")
    return ids
generate_random_user_ids(1000, 1500)

random user id used while in development



array([532439, 532649, 532710, 533381, 534018, 534046, 534152, 534777,
       535396, 536526, 537150, 538373, 538741, 539019, 539354, 539743,
       539791, 540794, 540807, 541826, 542027, 542345, 543865, 544022,
       544496, 544571, 544832, 544833, 545026, 545639, 546933, 547732,
       548064, 550170, 550777, 551208, 551817, 552356, 552997, 553632,
       553668, 553931, 555509, 555955, 556045, 556249, 556534, 556633,
       557801, 558634, 559021, 560851, 561184, 561923, 563237, 563607,
       563962, 564660, 565041, 565401, 565510, 565962, 568930, 569099,
       569386, 570522, 571700, 571702, 572124, 573330, 573434, 573537,
       573975, 573995, 574778, 574843, 575714, 575830, 575867, 576584,
       576763, 576990, 577547, 578237, 578632, 578699, 580784, 581199,
       582829, 582996, 584542, 584750, 584893, 586421, 586652, 586741,
       587259, 587685, 588344, 589596, 589626, 589682, 589967, 590113,
       590466, 591072, 591866, 592003, 592122, 592281, 592455, 592671,
      

### Details about a specific customer

In [205]:
# ids I like -> 28812, 56514, 56520, 1488844
customer_id_use = 534046 #84100

In [206]:
#all_custumers_id_rows(customer_id=customer_id_use)
## same as line above but including movie_year and movie_title

#all_customers_id_plus_movie_title_rows(customer_id=customer_id_use)
information.all_id_rows(df=data_rating_plus_movie_title, type="customer_id", item_id=customer_id_use)

,movie_id,customer_id,rating,movie_year,movie_title
3131,5,534046,5.0,2004.0,The Rise and Fall of ECW
8700,30,534046,5.0,2003.0,Something's Gotta Give
12247,52,534046,5.0,2002.0,The Weather Underground
31214,187,534046,3.0,2002.0,Death to Smoochy
34441,194,534046,4.0,1996.0,Arliss: The Best of Arliss
51683,290,534046,4.0,2004.0,Harold and Kumar Go to White Castle
111420,571,534046,4.0,1999.0,American Beauty
187786,1029,534046,1.0,1995.0,Tromeo and Juliet
210575,1148,534046,4.0,1981.0,For Your Eyes Only
289148,1642,534046,5.0,1995.0,Casino: 10th Anniversary Edition


In [207]:
#get_users_avg_rating(customer_id=customer_id_use)
information.get_item_avg_rating(df=all_customers_average_ratings, type='customer_id', item_id=customer_id_use)

customer_id rating       avg_rating
                    sum count           
1005      534046  113.0    28   4.035714

In [208]:
#display_movies_customer_rated_higher_than(customer_id=customer_id_use, min_rating=4)
information.display_movies_customer_rated_higher_than(df=data_rating_plus_movie_title, customer_id=customer_id_use, min_rating=min_rating)

                                  movie_title  rating
movie_id                                             
5                    The Rise and Fall of ECW     5.0
30                     Something's Gotta Give     5.0
52                    The Weather Underground     5.0
194                Arliss: The Best of Arliss     4.0
290       Harold and Kumar Go to White Castle     4.0
571                           American Beauty     4.0
1148                       For Your Eyes Only     4.0
1642         Casino: 10th Anniversary Edition     5.0
1719       The Life Aquatic with Steve Zissou     5.0
1798                            Lethal Weapon     4.0
1843                    Beverly Hills Cop III     4.0
2178                                     Lock     4.0
2464                                 Trekkies     4.0
2782                               Braveheart     4.0
2862                 The Silence of the Lambs     5.0
2939              Michael Moore Hates America     5.0
3290                        

In [209]:
information.display_movies_customer_rated_lower_than(df= data_rating_plus_movie_title,customer_id=customer_id_use, max_rating=max_rating)

                  movie_title  rating
movie_id                             
187          Death to Smoochy     3.0
1029        Tromeo and Juliet     1.0
1700            Bottle Rocket     3.0
1744        Beverly Hills Cop     3.0
2840      Fantasm Comes Again     1.0
average rating 1005    4.035714
Name: avg_rating, dtype: float64


In [210]:
information.get_users_loved_hated_movies(df=data_rating_plus_movie_title, customer_id=customer_id_use, minmax_rating=4)

User 534046 loved these movies
The Rise and Fall of ECW
Something's Gotta Give
The Weather Underground
Arliss: The Best of Arliss
Harold and Kumar Go to White Castle
American Beauty
For Your Eyes Only
Casino: 10th Anniversary Edition
The Life Aquatic with Steve Zissou
Lethal Weapon
Beverly Hills Cop III
Lock
Trekkies
Braveheart
The Silence of the Lambs
Michael Moore Hates America
The Godfather
Lethal Weapon 3
Goldfinger
The People vs. Larry Flynt
Garden State
A Mighty Wind
Road to Perdition

and disliked these movies
Death to Smoochy
Tromeo and Juliet
Bottle Rocket
Beverly Hills Cop
Fantasm Comes Again


In [211]:
all_customers_avg_rating = all_customers_average_ratings["avg_rating"].mean()

usr = information.all_average_ratings(df=data_rating, type='customer_id')
usr = usr[usr["customer_id"] == 596533]

change = ((usr["avg_rating"] - all_customers_avg_rating) / all_customers_avg_rating)*100


print("average rating for user 596533:", float(usr["avg_rating"]))
print("average rating for all users:", all_customers_avg_rating)
print("difference between them", round(float(change), 2), "%")

average rating for user 596533: 4.571428571428571
average rating for all users: 3.5321404327111
difference between them 29.42 %


## Recommendations with collaborative filtering

#### functions

In [249]:
from surprise import dataset, KNNBaseline, accuracy
from surprise.model_selection import train_test_split, LeaveOneOut

def get_drop_list(type="movie_id"):
    #>IF MOVIE:  movie_id, movie rating count, movie rating mean
    #>IF CUSTOMER: customer_id, custumer rating count, customer rating mean
    df_count_mean_summary = data_rating.groupby(type)['rating'].agg(['count', 'mean'])
    df_count_mean_summary.index = df_count_mean_summary.index.map(int)
    #>IF MOVIE: returns 1799.0 as a benchmark number
    #IF CUSTOMER: returns 52.0 as a benchmark number
    benchmark = round(df_count_mean_summary['count'].quantile(0.7),0)
    # drop all rows below benchmark
    df_drop_list = df_count_mean_summary[df_count_mean_summary['count'] < benchmark]
    # return all indexes to drop
    return df_drop_list

def get_customer_recommendations(customer_id, predictor):
    #> returns movie_id, movie_year, movie_title
    chosen_customer_pred = data_movies.copy()
    # fails if movie_id is the index so we have to reset the index back to normal (0-N)
    chosen_customer_pred = chosen_customer_pred.reset_index()
    # makes sure that we only pick movies that are not in the movie dropped list
    chosen_customer_pred = chosen_customer_pred[~chosen_customer_pred['movie_id'].isin(df_movie_drop_list)]
    # make prediction for customer with id = <customer_id> and put it into 'estimated_score'
    chosen_customer_pred['estimated_score'] = chosen_customer_pred['movie_id'].apply(lambda x: predictor.predict(customer_id, x).est)
    # sort by 'estimated score'
    chosen_customer_pred = chosen_customer_pred.sort_values('estimated_score', ascending=False).set_index('movie_id')
    return chosen_customer_pred

# print out movies/tv show user has previously rated
def display_rated_content(customer_id=customer_id_use, number_to_show=20):
    print("Movies/TV Shows rated by customer", customer_id)
    #df = all_customers_id_plus_movie_title_rows(customer_id=customer_id)
    df = information.all_id_rows(df=data_rating_plus_movie_title, type='customer_id', item_id=customer_id)
    df = df[['movie_title', 'rating']].sort_values('rating', ascending=False)
    print(df[0:number_to_show].set_index('movie_title'))

def display_customers_recommendations(customer_id=customer_id_use, df=[], number_to_show=20):
    print("Movies/TV Shows recommended to customer")
    tmp_df = df[['movie_title', 'estimated_score']][0:number_to_show]
    tmp_df = tmp_df.set_index('movie_title')
    print(tmp_df)

def display_recommendation(customer_id, number_to_show, predictor):
    chosen_customer_pred = get_customer_recommendations(customer_id=customer_id, predictor=predictor)
    display_customers_recommendations(customer_id=customer_id, df=chosen_customer_pred, number_to_show=number_to_show)

def print_evaluation_accuracy(predictions):
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(predictions, verbose=False))
    print("MSE: ", accuracy.mse(predictions, verbose=False))
    print("MAE: ", accuracy.mae(predictions, verbose=False))
    # FCP = Fraction of Concordant Pairs
    print("FCP: ", accuracy.mae(predictions, verbose=False))

#### create recommenders

In [213]:
## creating a list of the movies that we don't wanna include
df_movie_drop_list = get_drop_list(type="movie_id")
#df_customer_drop_list = get_summary(type="customer_id")

In [250]:
def test_against_our_custom_function():
    _, data_rating, _, _ = NetflixLoadData.get_data_files(use_small_dataset=True)
    dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
    fullTrainset = dataset.build_full_trainset()

    trainSet, testSet = train_test_split(dataset, test_size=.25, random_state=1)

    algo = SVD(random_state=10)
    algo.fit(trainSet)

    predictions = algo.test(testSet)
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(predictions, verbose=False))
    print("MSE: ", accuracy.mse(predictions, verbose=False))
    print("MAE: ", accuracy.mae(predictions, verbose=False))
    # FCP = Fraction of Concordant Pairs
    print("FCP: ", accuracy.mae(predictions, verbose=False))


class convert_to_raw_ratings(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['customer_id'], df['movie_id'], df['rating'])]
        self.reader=reader

raw_ratings = convert_to_raw_ratings(data_rating, reader)
print("\nBuilding recommendation model...")
trainSet, testSet = train_test_split(raw_ratings, test_size=.25, random_state=1)

algo = SVD(random_state=10)
algo.fit(trainSet)
algo_predictions = algo.test(testSet)


Building recommendation model...


In [216]:
# save_to_pickle("algo_svd", algo)
# #save_to_pickle("algo_raw_rating", raw_ratings)
# save_to_pickle("algo_trainSet", trainSet)
# save_to_pickle("algo_testSet", testSet)

In [156]:
#cross_validate(algo, raw_ratings, measures=['MSE', 'RMSE', 'MAE'], cv=5, verbose=True)

In [157]:
svd = SVD(random_state=10)
data_new = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = data_new.build_full_trainset()
svd.fit(trainset)
svd_predictions = svd.test(testSet)


In [158]:
#cross_validate(svd, data_new, measures=['MSE', 'RMSE', 'MAE'], cv=5, verbose=True)

In [159]:
#surprise.get_neighbours(1, 10)

In [160]:
##TODO: look into knows_user 
#       -> which tells if the user is part of the trainset,
#           -> equals true if user has at least one rating

### display recommendations

In [161]:
number_to_show = 10

#### customer = 79724

In [162]:
display_rated_content(customer_id=79724, number_to_show=number_to_show)

Movies/TV Shows rated by customer 79724
Empty DataFrame
Columns: [rating]
Index: []


In [163]:
display_recommendation(customer_id=79724, number_to_show=number_to_show, predictor=svd)

Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
Lost: Season 1                                             4.521422
The Simpsons: Season 6                                     4.369304
The Godfather                                              4.355742
Six Feet Under: Season 4                                   4.350956
The Silence of the Lambs                                   4.326521
Lord of the Rings: The Fellowship of the Ring              4.285705
Law & Order: Special Victims Unit: The Second Year         4.252380
The Simpsons: Season 3                                     4.203618
The West Wing: Season 3                                    4.195865
Nip/Tuck: Season 2                                         4.181070


In [164]:
display_recommendation(customer_id=79724, number_to_show=number_to_show, predictor=algo)

Movies/TV Shows recommended to customer
                                               estimated_score
movie_title                                                   
Lost: Season 1                                        4.542424
The Simpsons: Season 6                                4.391883
The Godfather                                         4.340253
Six Feet Under: Season 4                              4.320674
Lord of the Rings: The Fellowship of the Ring         4.278653
Family Guy: Freakin' Sweet Collection                 4.260342
The Simpsons: Treehouse of Horror                     4.247825
The Simpsons: Season 3                                4.246159
The Silence of the Lambs                              4.240932
Nip/Tuck: Season 2                                    4.239350


#### customer = customer_id_use

In [165]:
display_rated_content(customer_id=customer_id_use, number_to_show=number_to_show)

Movies/TV Shows rated by customer 534046
                                    rating
movie_title                               
The Rise and Fall of ECW               5.0
Casino: 10th Anniversary Edition       5.0
A Mighty Wind                          5.0
The Godfather                          5.0
Michael Moore Hates America            5.0
The Silence of the Lambs               5.0
Something's Gotta Give                 5.0
The Life Aquatic with Steve Zissou     5.0
Road to Perdition                      5.0
The Weather Underground                5.0


In [166]:
display_recommendation(customer_id=customer_id_use, number_to_show=number_to_show, predictor=svd)

Movies/TV Shows recommended to customer
                                   estimated_score
movie_title                                       
The Silence of the Lambs                  4.975386
Six Feet Under: Season 4                  4.920907
Curb Your Enthusiasm: Season 3            4.874172
The Simpsons: Season 6                    4.871290
The Godfather                             4.827125
The Simpsons: Season 3                    4.770718
The Life of Birds                         4.757527
The Life of Mammals                       4.720321
The Simpsons: Treehouse of Horror         4.706994
The Simpsons: Season 1                    4.687082


In [167]:
display_recommendation(customer_id=customer_id_use, number_to_show=number_to_show, predictor=algo)
# recommends for example: Reservoir Dogs, which the user has already rated

Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
The Simpsons: Season 6                                     5.000000
Lord of the Rings: The Fellowship of the Ring              5.000000
The Simpsons: Season 3                                     5.000000
The Silence of the Lambs                                   5.000000
The Godfather                                              4.965630
The Twilight Zone: Vol. 3                                  4.920416
Lost: Season 1                                             4.917772
Law & Order: Special Victims Unit: The Second Year         4.902102
The Twilight Zone: Vol. 42                                 4.847656
Aliens: Collector's Edition                                4.846262


### print accuracy

In [168]:
print_evaluation_accuracy(svd_predictions)


Evaluating accuracy of model...
RMSE:  0.670030900148855
MAE:  0.5221479828726231


In [252]:
print_evaluation_accuracy(algo_predictions)


Evaluating accuracy of model...
RMSE:  0.9048402795139483
MSE:  0.8187359314308801
MAE:  0.7014346782230714
FCP:  0.7014346782230714


In [251]:
test_against_our_custom_function() #for algo predictions


Evaluating accuracy of model...
RMSE:  0.9048402795139483
MSE:  0.8187359314308801
MAE:  0.7014346782230714
FCP:  0.7014346782230714


results are the same so we will not use the custom function since it adds no value

## other

In [225]:

dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = dataset.build_full_trainset()

In [226]:
trainset

In [227]:
tmp_data_movies = data_movies[['movie_id', 'movie_title']]
tmp_data_movies = tmp_data_movies.set_index('movie_id').T
tmp_data_movies = tmp_data_movies.to_dict('list')
tmp_data_movies = {k: str(v[0]) for k,v in tmp_data_movies.items()}

In [228]:
similarity_matrix = KNNBasic(sim_options={'name': 'cosine', 'user_based': False})
similarity_matrix.fit(trainset).compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


array([[1.        , 0.97640822, 0.95372922, ..., 0.93954011, 0.81705717,
        0.9767252 ],
       [0.97640822, 1.        , 0.97627206, ..., 0.94840141, 0.9486833 ,
        0.98713529],
       [0.95372922, 0.97627206, 1.        , ..., 0.94160414, 0.92205571,
        0.93787593],
       ...,
       [0.93954011, 0.94840141, 0.94160414, ..., 1.        , 0.95197868,
        0.93313613],
       [0.81705717, 0.9486833 , 0.92205571, ..., 0.95197868, 1.        ,
        0.67445327],
       [0.9767252 , 0.98713529, 0.93787593, ..., 0.93313613, 0.67445327,
        1.        ]])

In [229]:
# generate_random_user_ids(1000, 1500)

In [230]:
# for i in generate_random_user_ids(1000, 1500):
#     if (len(all_custumers_id_rows(i)) > 3):
#         print(i)

In [231]:
test_subject = 596533#16272
k = 10

In [232]:
test_subject_iid = trainset.to_inner_uid(test_subject)

test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbours = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [233]:
candidates = defaultdict(float)

for itemID, rating in k_neighbours:
    try:
        similarities = similarity_matrix[itemID]
        for innerID, score in enumerate(similarities):
            candidates[innerID] += score * (rating / 5.0)
    except:
        continue

In [234]:
def getMovieName(movieID):
  if int(movieID) in tmp_data_movies:
    return tmp_data_movies[int(movieID)]
  else:
      return ""

In [235]:
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
    watched[itemID] = 1

recommendations = []
position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
        position += 1
        # only want top n which in our case in 10
        if(position > 10): break

for rec in recommendations:
    print("Movie: ", rec)


In [236]:
print("user", test_subject, 'previous recommendations')
display_recommendation(customer_id=test_subject, number_to_show=10, predictor=svd)

user 596533 previous recommendations
Movies/TV Shows recommended to customer
                                          estimated_score
movie_title                                              
Lost: Season 1                                   5.000000
Fight Club: Bonus Material                       5.000000
The Silence of the Lambs                         4.982024
The Simpsons: Season 6                           4.964567
The Godfather                                    4.934454
The Twilight Zone: Vol. 12                       4.925539
Nip/Tuck: Season 2                               4.912042
Six Feet Under: Season 4                         4.910300
House of Cards Trilogy I: House of Cards         4.907765
The West Wing: Season 3                          4.890468


In [237]:
#TODO:
#for the new user problem
## find highest rated movie / tv shows that have been watched the most -> how likely is new user to click on it ?

In [238]:
#cross_validate(similarity_matrix, dataset, measures=['MSE', 'RMSE', 'MAE'], cv=5, verbose=True)


Evaluating accuracy of model...
RMSE:  0.9048402795139483
MSE:  0.8187359314308801
MAE:  0.7014346782230714
FCP:  0.7014346782230714
